# 1.Data Import and Combination

In [8]:
import pandas as pd
import gensim as gs
import nltk
import re
from nltk.corpus import stopwords
import sklearn as sk
import numpy as np
import pickle
from tqdm import tqdm

## 1.1.import dataset

In [9]:
train_path="DataSet\\train.txt"
test_path="DataSet\\test.txt"

train_df = pd.read_csv(train_path,sep="\t",index_col=0)
test_df=pd.read_csv(test_path,sep="\t",index_col=0)


In [10]:
train_df

,turn1,turn2,turn3,label
id,,,,
0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,By,by Google Chrome,Where you live,others
3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,Just for time pass,wt do u do 4 a living then,Maybe,others
...,...,...,...,...
30155,I don't work,I could take your shift,I am a student,others
30156,I'm not getting you 😭😭😭,Why are you crying??,Because you are not making any sense,sad
30157,Haha,"no, seriously. What is up with that o-o",Had your breakfast?,others


In [11]:
test_df

,turn1,turn2,turn3,label
id,,,,
0,Hmm,What does your bio mean?,I don’t have any bio,others
1,What you like,very little things,Ok,others
2,Yes,How so?,I want to fuck babu,others
3,what did you guess,what what,fuck,others
4,We ?,of course we will!,What gender movies you like??,others
...,...,...,...,...
5504,Not youuu,I also didn't not not.,How to calll,others
5505,Welcome,"Why, thank you.","I don't know, you tell",others
5506,Yes,IF ONLY I COULD AFFORD THIS,How are you,others


## 1.2.unify the variables for change

In [12]:
# now when I copy the code for another dataset, I can easily change the special name of the dataset for adjusting the process and storage
process_df = train_df
version_change = "glove"
path_change = "train"

In [13]:
emo_path="DataSet\\emo_collection_{b}.csv".format(b=version_change)
emo_df=pd.read_csv(emo_path,sep="|")

In [14]:
emo_df

,emoji/emoticon,key words
0,😀,face grin grinning face
1,😃,face grinning face big eyes mouth open smile
2,😄,eye face grinning face smiling eyes mouth ope...
3,😁,beaming face smiling eyes eye face grin smile
4,😆,face grinning squinting face laugh mouth sati...
...,...,...
2279,( ´Д｀)ﾉ(´･ω･`) ﾅﾃﾞﾅﾃﾞ,patting made made
2280,(*ﾟﾉOﾟ)<ｵｵｵｵｫｫｫｫｫｫｫｰｰｰｰｰｲ!,calling ooooh
2281,( ﾟ∀ﾟ)ｱﾊﾊ八八ﾉヽﾉヽﾉヽﾉ ＼ / ＼/ ＼,evil laugh literally ahahahaha
2282,（・∀・ ）ヾ(- -；)コラコラ,blaming


## 1.3.get label and turn into category

In [15]:
# set the label into categorical number:
# others:0; happy:1,angry:3;sad:2;

class Label_format:
    def __init__(self,df):
        self.df=df
        
    def get_y(self):
        y = self.df.iloc[:,-1].values
        return y
    
    def label_category(self,y):
        change_list = list()
        emotion = {"others":0,"happy":1,"sad":2,"angry":3}
        for each in y:
            change_list.append(emotion[each])
        change_array=np.array(change_list)
        return change_array
    
    def save_file(self,data,name):
        pickle_file = open('DataSet\\'+name,mode='wb')
        pickle.dump(data,pickle_file)
        pickle_file.close()

In [16]:
#set y label, and we make X later on.

lf = Label_format(process_df)
y = lf.get_y()
y = lf.label_category(y)
save_path = "{a}_y.pkl".format(a=path_change)
lf.save_file(y,save_path)

In [17]:
y

array([0, 3, 0, ..., 0, 0, 0])

## 1.4. class for combination

In [11]:
# because we use pre-trained embeddings, we can't directly use stemming, remove stopwords or remove punctuations.
# we need to turn the data as much as possible close to the pre-trained embedding.
def basic_clean_text(input_text):
    'clean text function'
    #clean emoji, punctions
    input_text = re.sub(r'[^a-zA-Z]',repl=" ",string=input_text)
    #lower the words
    input_text =input_text.lower()
    #stemming the words
    list_text=input_text.split()
    ps=nltk.stem.porter.PorterStemmer()
    list_text_out=list()
    for word in list_text:
        if word in set(stopwords.words('english')):
            continue
        list_text_out.append(ps.stem(word))
    input_text=" ".join(list_text_out) 
    return input_text

def combine_method(df):
    turn1_corpus= list(df["turn1"])
    turn2_corpus= list(df["turn2"])
    turn3_corpus= list(df["turn3"])
    
    # use text_content as dict for storing the data of 
    text_content={"turn1":turn1_corpus,"turn2":turn2_corpus,"turn3":turn3_corpus}
    
    # because we need all the sentences including sentences in turn1, turn2, turn3 
    # to predict the emtion situation, therefore we need to combine them together
    length = len(text_content["turn1"])
    text_all_content = list()
    for i in tqdm(range(length)):
        text_row_all = text_content["turn1"][i] +" " + text_content["turn2"][i] +" "+ text_content["turn3"][i]
        text_all_content.append(text_row_all)
    text_all_content = {"all":text_all_content}
    
    return text_content,text_all_content

In [12]:
text_3,text_all = combine_method(process_df)

100%|███████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 1042750.29it/s]


In [13]:
text_3

{'turn1': ["Don't worry  I'm girl",
  'When did I?',
  'By',
  'U r ridiculous',
  'Just for time pass',
  "I'm a dog person",
  'So whatsup',
  'Ok',
  'Really?',
  'Bay',
  'I hate my boyfriend',
  'I will do night.',
  'Sure go ahead',
  'Bad',
  'Ok get it......',
  'Money money and lots of money😍😍',
  'My gf left ne',
  'get lost',
  'You are lying and i know that',
  'Ur creator is very bad',
  'ehat is hehe',
  'Do you dance?',
  'I hate it too',
  'Not always',
  'Bcoz u dont know wat is to miss someone',
  'Yeah',
  'I want to tell you something',
  'Yes I will send the email and you write all yours filling still I write all matters but your are clever and not any matter for yourself please send andwrite',
  'You are very funny',
  'wth',
  'so rude',
  'ok i will',
  'Ntng.....',
  'You are fool',
  'U dare me?',
  'Abt me',
  'crazy',
  'Goa in India',
  'That was mean',
  'yes :)',
  "I don't no",
  'Send me your nekde pic',
  "OK friend's",
  'She is ignoring me',
  'Happy

In [14]:
text_all

{'all': ["Don't worry  I'm girl hmm how do I know if you are What's ur name?",
  'When did I? saw many times i think -_- No. I never saw you',
  'By by Google Chrome Where you live',
  'U r ridiculous I might be ridiculous but I am telling the truth. U little disgusting whore',
  'Just for time pass wt do u do 4 a living then Maybe',
  "I'm a dog person youre so rude Whaaaat why",
  'So whatsup Nothing much. Sitting sipping and watching TV. How abt u? What are you watching on tv?',
  'Ok ok im back!! So, how are u',
  'Really? really really really really really Y saying so many times...i can hear you',
  'Bay in the bay 😘 love you',
  'I hate my boyfriend you got a boyfriend? Yes',
  'I will do night. Alright. Keep me in loop. Not giving WhatsApp no.',
  'Sure go ahead Many thanks once again! Love you too',
  "Bad Bad bad! That's the bad kind of bad. I have no gf",
  'Ok get it...... I made it an option Ok',
  "Money money and lots of money😍😍 I need to get it tailored but I'm in love w

## 1.5. Main Function

In [15]:
def fit_transform(input_dict,method):
    output_dict=dict()
    for key in input_dict.keys():
        processed_texts = list()
        for text in tqdm(input_dict[key]):
            processed_single_text = eval(method)(text)
            processed_texts.append(processed_single_text)
        
        output_dict[key]=processed_texts
    
    return output_dict

# 2. Preprocessing for cleaning the data
## 2.1. Extract emoji, emoticons based on emo_df, and build 2 versions of data
- one version for saving the extracted emoji, emoticons of each sentence
- one version for text without the emoji, emoticons based on emo_df

In [16]:
emo_values = list(emo_df["emoji/emoticon"])
meaning_values = list(emo_df["key words"])

def count_emo(sentence,emo):
    'function for counting how many same emoji or emoticon in one sentence'
    sentence = sentence.replace(emo," count_no ")
    words_list = sentence.split()
    count_no=0
    for word in words_list:
        if "count_no" in word:
            count_no +=1
    
    return count_no
            

def extract_emo(text_dict,emo_values,meaning_values):
    calculate_emo=dict()
    del_emo_dict=dict()
    for key in text_dict.keys():
        length = len(text_dict[key])
        del_emo_list = text_dict[key].copy()
        
        # set list_all_emo as the storage for storing all emoji and emoticons for all sentences
        list_all_emo = list()

        len_emo = len(emo_values)
        for i in tqdm(range(length)):
            
            # set list_emo as the storage for storing the emo for each sentence
            list_emo = list()
            for e in range(len_emo):
                if emo_values[e] in text_dict[key][i]:
                    count_no = count_emo(text_dict[key][i],emo_values[e])
                    list_emo.append([count_no,meaning_values[e]])
                    del_emo_list[i]=del_emo_list[i].replace(emo_values[e], " ")
            
            list_all_emo.append(list_emo)

        calculate_emo[key]=list_all_emo
        del_emo_dict[key]=del_emo_list
    
    return del_emo_dict,calculate_emo


In [17]:
# save the storage of emoji and emoticons
def save_data(name,data):
    pickle_file = open('DataSet\\'+name,mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [18]:
text_3_0, calculate_emo_3 = extract_emo(text_3,emo_values,meaning_values)
text_all_0, calculate_emo_all = extract_emo(text_all,emo_values,meaning_values)

100%|██████████████████████████████████████████████████████████████████████████| 30160/30160 [00:06<00:00, 4444.54it/s]


In [19]:
calculate_emo_3

{'turn1': [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[2, ' eye face love smile smiling face heart eyes ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' smiley happy face ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' tongue sticking cheeky playful blowing raspberry ']],
  [],
  [[1, ' unamused face '], [1, ' cold downcast face sweat face sweat ']],
  [],
  [[2, ' face face tears joy joy laugh tear ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' break broken broken heart ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' face face steam nose triumph won ']],
  [],
  [],
  [],
  [],
  [[1, ' face grinning face big eyes mouth open smile ']],
  [],
  [[1, ' cat cat wry sm

In [20]:
save_path = "extraction_emo_3_{a}_{b}.pkl".format(a=path_change,b=version_change)
save_data(save_path,calculate_emo_3)

In [21]:
text_3_0

{'turn1': ["Don't worry  I'm girl",
  'When did I?',
  'By',
  'U r ridiculous',
  'Just for time pass',
  "I'm a dog person",
  'So whatsup',
  'Ok',
  'Really?',
  'Bay',
  'I hate my boyfriend',
  'I will do night.',
  'Sure go ahead',
  'Bad',
  'Ok get it......',
  'Money money and lots of money  ',
  'My gf left ne',
  'get lost',
  'You are lying and i know that',
  'Ur creator is very bad',
  'ehat is hehe',
  'Do you dance?',
  'I hate it too',
  'Not always',
  'Bcoz u dont know wat is to miss someone',
  'Yeah',
  'I want to tell you something',
  'Yes I will send the email and you write all yours filling still I write all matters but your are clever and not any matter for yourself please send andwrite',
  'You are very funny',
  'wth',
  'so rude',
  'ok i will',
  'Ntng.....',
  'You are fool',
  'U dare me?',
  'Abt me',
  'crazy',
  'Goa in India',
  'That was mean',
  'yes  ',
  "I don't no",
  'Send me your nekde pic',
  "OK friend's",
  'She is ignoring me',
  'Happy'

# 2.2. Lower the words

In [22]:
# lower the words
def lower_text(input_text):
    #lower the words
    input_text =input_text.lower()
    return input_text

In [23]:
text_3_1 = fit_transform(text_3_0,"lower_text")
text_all_1 = fit_transform(text_all_0,"lower_text")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 165224.33it/s]


## 2.3. Seperate the adhered words with space and remove redundant emoji, emoticons
 - e.g. "you?" ? and you are adhered together, which needs to be split
 - e.g. "yess i am crazyyy😂😂" I also need to split emoji from the word "crazyyy" as "i am crazyyy 😂 😂 "
 - and some emoji, emoticons can't be found in the emo_df, therefore, I delete them since it's impossible to explain them in word2vec

In [24]:
# functions for extracting emoji, emoticons and punctuations
def extract_emoji(input_text):
    emoji_list = re.findall(r'(?:[^a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',input_text)
    return emoji_list
    
def extract_emoticon(input_text):
    emoticon_list = re.findall(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',input_text)
    return emoticon_list

def extract_punctuation(input_text):
    'here I use for loop to split the double punctuations into single ones'
    punctuation_list = re.findall(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',input_text)
    result_list=list()
    for each in punctuation_list:
        list_split=[each]
        if each !="...":
            list_split=list(each)
        for punc in list_split:
            result_list.append(punc)
    return result_list

# functions for assigning emoji, emoticons and punctuations space from each other
def assign_space(input_text):
    #emoji
    input_text = re.sub(r'(?:[^ a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\,\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',' ',input_text)
    #emoticon
    input_text = re.sub(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',' ',input_text)
    #punctuation
    input_text = re.sub(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',' \g<0> ' ,input_text)
    #replace the redundant space
    input_text = re.sub(r'[a-zA-Z]+',r' \g<0> ',input_text)
    input_text = re.sub(r'  +',r' ',input_text)
    return input_text

In [25]:
text_3_2 = fit_transform(text_3_1,"assign_space")
text_all_2 = fit_transform(text_all_1,"assign_space")

100%|█████████████████████████████████████████████████████████████████████████| 30160/30160 [00:01<00:00, 25957.00it/s]


# 3. Preprocessing: Correct the words in sentence
## 3.1. Correct common words

In [26]:
# turn don't, doesn't, can't, couldn't, shouldn't, didn't, isn't, wasn't, weren't, aren't, won't, shan't, 's, 're
# as do not, do not, can not, could not, should not, did not, is not, was not, were not, are not, will not, shall not, 's, are

# turn r , u, bt, n, 
# as are, you ,but, and
cor_dict= {
    "don ' t":      "do not",
    "doesn ' t":    "do not",
    "can ' t":      "can not",
    "couldn ' t":   "could not",
    "shouldn ' t":  "should not",
    "didn ' t":     "did not",
    "isn ' t":      "is not",
    "wasn ' t":     "was not",
    "weren ' t":    "were not",
    "aren ' t":     "are not",
    "won ' t":      "will not",
    "shan ' t":     "shall not",
    
    #"\' re ":       "are ",
    #" re ":         "are",
    "\' m ":        "am ",
    "\' ll ":       "will ",
    #" \' s ":       " \'s "

    }

In [27]:
def correct_words_common(input_text):
    'for correcting the words in sentences based on hand-crafted data'
    corrected_sentence = input_text
    for each_key in cor_dict.keys():
        if each_key in input_text:
            corrected_sentence = corrected_sentence.replace(each_key,cor_dict[each_key])
    return corrected_sentence


In [28]:
text_3_3 = fit_transform(text_3_2,"correct_words_common")
text_all_3 = fit_transform(text_all_2,"correct_words_common")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 119528.58it/s]


## 3.2. Correct missing characters
- e.g. "hmmm i can talk no w", here "now" is missing "w".

In [29]:
def correct_words_combin(input_text):
    corrected_sentence = input_text
    corrected_sentence = re.sub(r' ([a-z]+) ([bcdefghjklnopqrstvwxz]) ',r' \g<1>\g<2> ',corrected_sentence)
    return corrected_sentence

In [30]:
testset = {"turn1":["I ' m go d , and I wasn ' t player wai t ", "hhh you ' re kidding m e "]}
print(fit_transform(testset,"correct_words_combin"))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]

{'turn1': ["I ' m god , and I wasn ' t player wait ", "hhh you ' re kidding me "]}


In [31]:
text_3_4 = fit_transform(text_3_3,"correct_words_combin")
text_all_4 = fit_transform(text_all_3,"correct_words_combin")

100%|█████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 78751.66it/s]


In [32]:
text_all_4["all"][:10]

[" do not worry i am girl hmm how do i know if you are what ' s ur name ?",
 ' when did i ? saw many times i thinkn o . i never saw you ',
 ' by by google chrome where you live ',
 ' ur ridiculous i might be ridiculous but i am telling the truth . u little disgusting whore ',
 ' just for time pass wt do u do 4 a living then maybe ',
 ' i am a dog person youre so rude whaaaat why ',
 ' so whatsup nothing much . sitting sipping and watchingt v . how abt u ? what are you watching on tv ?',
 ' ok ok im back so , how are u ',
 ' reall y ? really really really really really y saying so many times i can hear you ',
 ' bay in the bay love you ']

## 3.3. Remove punctuations

In [33]:
def remove_punc(input_text):
    new_sentence = input_text
    new_sentence = re.sub(r'[^ a-zA-Z]',"",new_sentence).strip()
    return new_sentence

In [34]:
text_3_5 = fit_transform(text_3_4,"remove_punc")
text_all_5 = fit_transform(text_all_4,"remove_punc")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 121433.98it/s]


In [35]:
text_all_5["all"][:10]

['do not worry i am girl hmm how do i know if you are what  s ur name',
 'when did i  saw many times i thinkn o  i never saw you',
 'by by google chrome where you live',
 'ur ridiculous i might be ridiculous but i am telling the truth  u little disgusting whore',
 'just for time pass wt do u do  a living then maybe',
 'i am a dog person youre so rude whaaaat why',
 'so whatsup nothing much  sitting sipping and watchingt v  how abt u  what are you watching on tv',
 'ok ok im back so  how are u',
 'reall y  really really really really really y saying so many times i can hear you',
 'bay in the bay love you']

## 3.4.  Remove stopwords made handcrafted

In [36]:
print(stopwords.words('english')) # which is not suitable here, coz e.g. in emotion analysis, not happy is opposite from happy.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [37]:
stopwords_byhand = []
# since combination of words can make the sentiment totally different
# e.g. "wasn't happy" is more likely to be sad, not happy.
# e.g. "you're sad" doesn't mean "I'm sad"
# therefore in order not to mix the order of the sentence, I try to keep those stopwords in sentence
# besides, when doing CNN or LSTM, it's important to detect all the structure of the sentence

In [38]:
def remove_stopwords(input_text):

    words = input_text.split()
    new_words=list()
    for word in words:
        if word in stopwords_byhand:
            continue
        else:
            new_words.append(word)
    sentence = " ".join(new_words)
  
    return sentence

In [39]:
text_3_6 = fit_transform(text_3_5,"remove_stopwords")
text_all_6 = fit_transform(text_all_5,"remove_stopwords")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 111998.03it/s]


# 4. Preprocessing: Correction of Incorrect Spell and Shorthand Words
## 4.1. Correcting spell mistake
 - e.g. "nt" is actually "not", "noo" is actually "no"
 - in order to do so, here i use library called pyspellchecker

In [40]:
from spellchecker import SpellChecker
spell = SpellChecker()
def correct_spell(input_text):

    words = input_text.split()
    new_words=list()
    for word in words:
        word = spell.correction(word)
        new_words.append(word)
    sentence = " ".join(new_words)
    return sentence


In [41]:
text_3_7 = fit_transform(text_3_6,"correct_spell")
text_all_7 = fit_transform(text_all_6,"correct_spell")

100%|████████████████████████████████████████████████████████████████████████████| 30160/30160 [23:36<00:00, 21.29it/s]


In [42]:
text_all_7 ["all"][:10]

['do not worry i am girl hmm how do i know if you are what s ur name',
 'when did i saw many times i think o i never saw you',
 'by by goole chrome where you live',
 'ur ridiculous i might be ridiculous but i am telling the truth u little disgusting whore',
 'just for time pass wt do u do a living then maybe',
 'i am a dog person your so rude whaaaat why',
 'so whatsit nothing much sitting sipping and watching v how at u what are you watching on tv',
 'ok ok im back so how are u',
 'really y really really really really really y saying so many times i can hear you',
 'bay in the bay love you']

In [43]:
# after spell mistakce correction, it's needed to run over again to remove the stopwords
text_3_8 = fit_transform(text_3_7,"remove_stopwords")
text_all_8 = fit_transform(text_all_7,"remove_stopwords")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 117655.13it/s]


## 4.2. shorthand words translation
- here we only deal with the most frequent words happened in text which can't be changed into vectors based on pretrained word2vec index

In [84]:
cor_dict2= {
    "whatsaap":"whatsapp",
    "whatsappn":"whatsapp",
    "ohhhooo":"oh oh",
    "tel-me":"tell me",
    "hahahahhahh":"hahaha",
    "okkkkkkkk":"ok",
    "heeaaadd":"head",
    "happyhalloween":"happy halloween",
    "hahhhahh":"hahaha",
    "amazingggggggg":"amazing",
    "whtsapp":"whatsapp",
    "thankyouuuu":"thank you",
    }

In [85]:
def correct_words_common2(input_text):
    'for correcting the words in sentences based on hand-crafted data'
    corrected_sentence = " "+input_text+" "
    for each_key in cor_dict2.keys():
        if each_key in input_text:
            rule = " "+each_key+" "
            corrected_sentence = re.sub(rule," "+cor_dict2[each_key]+" ",corrected_sentence).strip()
            #corrected_sentence.replace(each_key,cor_dict2[each_key])
    return corrected_sentence

In [86]:
text_3_9 = fit_transform(text_3_8,"correct_words_common2")
text_all_9 = fit_transform(text_all_8,"correct_words_common2")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 133807.57it/s]


# 5. check data format 
- for further preprocessing based on pre-trained embedding resource

## 5.1. check distinct vocabulary

In [44]:
def distinct_words(dialogs):
    vocab = dict()
    for each_dialog in tqdm(dialogs):
        words_list = each_dialog.split(sep=" ")
        for word in words_list:
            if word == '':
                continue
            try:
                vocab[word] +=1
            except KeyError:
                vocab[word] = 1
    
    return vocab

In [45]:
vocab = distinct_words(text_all["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version original",b=len(vocab)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 256272.50it/s]

the length of distinct vocabulary of the version original is 33024


In [46]:
vocab0 = distinct_words(text_all_0["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version 0",b=len(vocab0)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 258465.41it/s]

the length of distinct vocabulary of the version 0 is 30589


In [47]:
vocab1 = distinct_words(text_all_1["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version 1",b=len(vocab1)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 272234.32it/s]

the length of distinct vocabulary of the version 1 is 25453


In [48]:
vocab2 = distinct_words(text_all_2["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 2",b=len(vocab2)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 236252.48it/s]

the number of distinct vocabulary of the version 2 of data is 15601


In [49]:
vocab3 = distinct_words(text_all_3["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 3",b=len(vocab3)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 249847.84it/s]

the number of distinct vocabulary of the version 3 of data is 15597


In [50]:
vocab4 = distinct_words(text_all_4["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 4",b=len(vocab3)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 258406.80it/s]

the number of distinct vocabulary of the version 4 of data is 15597


In [51]:
vocab5 = distinct_words(text_all_5["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 5",b=len(vocab5)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 274709.62it/s]

the number of distinct vocabulary of the version 5 of data is 15307


In [52]:
vocab6 = distinct_words(text_all_6["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 6",b=len(vocab6)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 288003.17it/s]

the number of distinct vocabulary of the version 6 of data is 15307


In [53]:
vocab7 = distinct_words(text_all_7["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 7",b=len(vocab7)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 277225.49it/s]

the number of distinct vocabulary of the version 7 of data is 11272


In [54]:
vocab8 = distinct_words(text_all_8["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 8",b=len(vocab8)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 288007.76it/s]

the number of distinct vocabulary of the version 8 of data is 11272


In [87]:
vocab9 = distinct_words(text_all_9["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 9",b=len(vocab9)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 272235.49it/s]

the number of distinct vocabulary of the version 9 of data is 11261


## 5.2. check the percentage of words in data can be processed by word2vec pre-trained data

In [55]:
# import processed words list from glove
GloVe_path="DataSet\\glove.840B.300d_words.pkl"
def open_pkl(path):
    pickle_file = open(path,mode='rb')
    data = pickle.load(pickle_file)
    pickle_file.close()

    return data

def form_dict(words_list):
    words_dict = dict()
    content="just for data check"
    for word in words_list:
        words_dict[word]=content
    
    return words_dict

In [56]:
glo_words=open_pkl(GloVe_path)
glo_dict = form_dict(glo_words)# because process dict is much faster than processing list

In [57]:
glo_dict["many"]

'just for data check'

In [58]:
# build the function for check the coverage of the vocabulary and the text
import operator
def check_coverage(vocab, embedding_resource):
    cov_vocab =0
    num_vocab = len(vocab)
    cov_text = 0
    not_cov = dict()
    not_text = 0
    for word in tqdm(vocab):
        try:
            x = embedding_resource[word]
            cov_vocab += 1
            cov_text += vocab[word]
        except:
            not_cov[word]=vocab[word]
            not_text += vocab[word]
            pass
    percent_cov_vocab = cov_vocab/num_vocab
    percent_cov_text = cov_text/(cov_text+not_text)
    print("In Embedding Index we have {:.2%} coverage of distinct vocabulary".format(percent_cov_vocab))
    print("And we have {:.2%} coverage of all text".format(percent_cov_text))
    sorted_not_cov = sorted(not_cov.items(),key= operator.itemgetter(1),reverse = True)
    print("The number of words which are not covered in word2vec resource is: {0}".format(len(sorted_not_cov)))
    return sorted_not_cov

In [59]:
# original version of data
not_cov = check_coverage(vocab, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 33024/33024 [00:00<00:00, 1102325.38it/s]

In Embedding Index we have 51.14% coverage of distinct vocabulary
And we have 90.12% coverage of all text
The number of words which are not covered in word2vec resource is: 16137


In [60]:
not_cov[:20]

[('you?', 651),
 ('😂', 601),
 (':(', 512),
 ('what?', 465),
 ("You're", 347),
 ("That's", 320),
 ('😂😂', 272),
 ('me?', 246),
 ("What's", 225),
 ('why?', 222),
 ('it?', 202),
 ('Why?', 194),
 ("what's", 186),
 ('😂😂😂', 177),
 ('you!', 176),
 ("i'm", 159),
 ('What?', 157),
 ('..', 153),
 ('😞', 145),
 ("won't", 141)]

In [61]:
# remove emoji, emoticon
not_cov0 = check_coverage(vocab0, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 30589/30589 [00:00<00:00, 1095397.82it/s]


In Embedding Index we have 55.24% coverage of distinct vocabulary
And we have 91.91% coverage of all text
The number of words which are not covered in word2vec resource is: 13692


In [62]:
not_cov0[:20]
# here we can see that the trained word2vec resource can't cover some stopwords, punctuations and the combination with word and punctuation

[('you?', 654),
 ('what?', 468),
 ("You're", 348),
 ("That's", 321),
 ('me?', 249),
 ("What's", 225),
 ('why?', 223),
 ('it?', 202),
 ('Why?', 194),
 ("what's", 186),
 ('you!', 176),
 ('..', 161),
 ("i'm", 159),
 ('What?', 157),
 ("won't", 141),
 ('u?', 140),
 ("Let's", 128),
 ('that?', 128),
 ('so?', 118),
 ('now?', 118)]

In [63]:
# lower the words
not_cov1 = check_coverage(vocab1, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 25453/25453 [00:00<00:00, 1063386.45it/s]

In Embedding Index we have 51.05% coverage of distinct vocabulary
And we have 91.48% coverage of all text
The number of words which are not covered in word2vec resource is: 12460


In [64]:
not_cov1[:20]
# here we can see that word2vec resource can't cover the words that are not correct spelled.

[("i'm", 2655),
 ('you?', 699),
 ('what?', 634),
 ('why?', 437),
 ("what's", 412),
 ("i'll", 360),
 ('me?', 260),
 ('it?', 207),
 ("let's", 195),
 ('you!', 186),
 ("i've", 164),
 ('..', 161),
 ("won't", 150),
 ('u?', 148),
 ('no,', 148),
 ('yes,', 138),
 ('so?', 132),
 ('that?', 128),
 ('now?', 123),
 ('not?', 105)]

In [65]:
# seperate adhered words such as "you!!!" with space into "you ! ! ! "
not_cov2 = check_coverage(vocab2, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 15601/15601 [00:00<00:00, 1196389.67it/s]

In Embedding Index we have 85.50% coverage of distinct vocabulary
And we have 99.42% coverage of all text
The number of words which are not covered in word2vec resource is: 2262


In [66]:
not_cov2[:20] #now we can see the adhered words pair such as "you!!!" are gone

[(',,', 23),
 ('yrr', 15),
 (',,,', 11),
 ('okh', 10),
 ('loudl', 10),
 ('ntng', 9),
 ('sarcasti', 8),
 ('bangaram', 7),
 ('ohhk', 7),
 ('bahubali', 7),
 ('byyy', 6),
 ('eriences', 6),
 ('artificia', 6),
 ('patienc', 6),
 (',?', 5),
 ('lanation', 5),
 ('adorabl', 5),
 ('wednesda', 5),
 ('sarcas', 5),
 ('disappointe', 5)]

In [67]:
# correct common words e.g. "you're" as "you are"
not_cov3 = check_coverage(vocab3, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 15597/15597 [00:00<00:00, 1202922.96it/s]

In Embedding Index we have 85.50% coverage of distinct vocabulary
And we have 99.41% coverage of all text
The number of words which are not covered in word2vec resource is: 2262


In [68]:
not_cov3[:20]

[(',,', 23),
 ('yrr', 15),
 (',,,', 11),
 ('okh', 10),
 ('loudl', 10),
 ('ntng', 9),
 ('sarcasti', 8),
 ('bangaram', 7),
 ('ohhk', 7),
 ('bahubali', 7),
 ('byyy', 6),
 ('eriences', 6),
 ('artificia', 6),
 ('patienc', 6),
 (',?', 5),
 ('lanation', 5),
 ('adorabl', 5),
 ('wednesda', 5),
 ('sarcas', 5),
 ('disappointe', 5)]

In [69]:
# add missing charactors e.g. "hmmm i can talk no w", here "now" is missing "w".
not_cov4 = check_coverage(vocab4, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 15400/15400 [00:00<00:00, 1286801.37it/s]

In Embedding Index we have 83.21% coverage of distinct vocabulary
And we have 99.28% coverage of all text
The number of words which are not covered in word2vec resource is: 2586


In [70]:
not_cov4[:20]

[('howr', 56),
 ('whyr', 33),
 (',,', 23),
 ('yrr', 15),
 ('snown', 12),
 ('notb', 11),
 ('goingo', 11),
 (',,,', 11),
 ('offt', 11),
 ('okh', 10),
 ('loudl', 10),
 ('ntng', 9),
 ('bangaram', 7),
 ('ohhk', 7),
 ('bahubali', 7),
 ('byyy', 6),
 ('hellr', 6),
 ('eriences', 6),
 ('needt', 6),
 ('hmmk', 6)]

In [71]:
# remove punctuation
not_cov5 = check_coverage(vocab5, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 15307/15307 [00:00<00:00, 1180817.19it/s]

In Embedding Index we have 83.30% coverage of distinct vocabulary
And we have 99.24% coverage of all text
The number of words which are not covered in word2vec resource is: 2556


In [72]:
not_cov5[:20]

[('howr', 56),
 ('whyr', 33),
 ('yrr', 15),
 ('snown', 12),
 ('notb', 11),
 ('goingo', 11),
 ('offt', 11),
 ('okh', 10),
 ('loudl', 10),
 ('ntng', 9),
 ('bangaram', 7),
 ('ohhk', 7),
 ('bahubali', 7),
 ('byyy', 6),
 ('hellr', 6),
 ('eriences', 6),
 ('needt', 6),
 ('hmmk', 6),
 ('shouldb', 6),
 ('whato', 6)]

In [73]:
# Remove stopwords made handcrafted
not_cov6 = check_coverage(vocab6, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 15307/15307 [00:00<00:00, 1278317.36it/s]

In Embedding Index we have 83.30% coverage of distinct vocabulary
And we have 99.24% coverage of all text
The number of words which are not covered in word2vec resource is: 2556


In [74]:
not_cov6[:20]

[('howr', 56),
 ('whyr', 33),
 ('yrr', 15),
 ('snown', 12),
 ('notb', 11),
 ('goingo', 11),
 ('offt', 11),
 ('okh', 10),
 ('loudl', 10),
 ('ntng', 9),
 ('bangaram', 7),
 ('ohhk', 7),
 ('bahubali', 7),
 ('byyy', 6),
 ('hellr', 6),
 ('eriences', 6),
 ('needt', 6),
 ('hmmk', 6),
 ('shouldb', 6),
 ('whato', 6)]

In [79]:
# correct misspell
not_cov7 = check_coverage(vocab7,glo_dict)

100%|████████████████████████████████████████████████████████████████████████| 11272/11272 [00:00<00:00, 807994.71it/s]

In Embedding Index we have 95.01% coverage of distinct vocabulary
And we have 99.84% coverage of all text
The number of words which are not covered in word2vec resource is: 563


In [80]:
not_cov7[:20]

[('pyy', 8),
 ('bahubali', 7),
 ('sinfield', 6),
 ('whatsaap', 4),
 ('mumby', 4),
 ('kalkara', 4),
 ('awsd', 4),
 ('uuuc', 3),
 ('useropenreflink', 3),
 ('wordstar', 3),
 ('bhabha', 3),
 ('whatsappn', 3),
 ('brabham', 3),
 ('ohhhooo', 3),
 ('tel-me', 2),
 ('bamhi', 2),
 ('tadybiar', 2),
 ('frndzzz', 2),
 ('kaleida', 2),
 ('vizagite', 2)]

In [81]:
# remove stopwords again after correction of missepll
not_cov8 = check_coverage(vocab8, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 11272/11272 [00:00<00:00, 1027250.88it/s]

In Embedding Index we have 95.01% coverage of distinct vocabulary
And we have 99.84% coverage of all text
The number of words which are not covered in word2vec resource is: 563


In [83]:
not_cov8[:50]

[('pyy', 8),
 ('bahubali', 7),
 ('sinfield', 6),
 ('whatsaap', 4),
 ('mumby', 4),
 ('kalkara', 4),
 ('awsd', 4),
 ('uuuc', 3),
 ('useropenreflink', 3),
 ('wordstar', 3),
 ('bhabha', 3),
 ('whatsappn', 3),
 ('brabham', 3),
 ('ohhhooo', 3),
 ('tel-me', 2),
 ('bamhi', 2),
 ('tadybiar', 2),
 ('frndzzz', 2),
 ('kaleida', 2),
 ('vizagite', 2),
 ('cnaa', 2),
 ('hahahahhahh', 2),
 ('okkkkkkkk', 2),
 ('padmavat', 2),
 ('bufi', 2),
 ('chweettt', 2),
 ('hawwww', 2),
 ('sampoornesh', 2),
 ('whatpc', 2),
 ('ingli', 2),
 ('heeaaadd', 2),
 ('happyhalloween', 2),
 ('emor', 2),
 ('multiverb', 2),
 ('awwwweee', 2),
 ('scuffletown', 2),
 ('widdup', 2),
 ('tuitalk', 2),
 ('foreheadwhen', 2),
 ('juraj', 2),
 ('hahhhahh', 2),
 ('amazingggggggg', 2),
 ('whtsapp', 2),
 ('thankyouuuu', 2),
 ('bsdi', 2),
 ('luhhhhhhh', 2),
 ('darras', 2),
 ('ayyyyyyyyyyyyyyy', 2),
 ('owwwwwww', 2),
 ('parsheen', 2)]

In [88]:
# correct shorthand words or some misspell which can't be corrected by pyspellchecker
not_cov9 = check_coverage(vocab9, glo_dict)

100%|████████████████████████████████████████████████████████████████████████| 11261/11261 [00:00<00:00, 705862.11it/s]

In Embedding Index we have 95.11% coverage of distinct vocabulary
And we have 99.85% coverage of all text
The number of words which are not covered in word2vec resource is: 551


In [89]:
not_cov9[:40]

[('pyy', 8),
 ('bahubali', 7),
 ('sinfield', 6),
 ('mumby', 4),
 ('kalkara', 4),
 ('awsd', 4),
 ('uuuc', 3),
 ('useropenreflink', 3),
 ('wordstar', 3),
 ('bhabha', 3),
 ('brabham', 3),
 ('bamhi', 2),
 ('tadybiar', 2),
 ('frndzzz', 2),
 ('kaleida', 2),
 ('vizagite', 2),
 ('cnaa', 2),
 ('padmavat', 2),
 ('bufi', 2),
 ('chweettt', 2),
 ('hawwww', 2),
 ('sampoornesh', 2),
 ('whatpc', 2),
 ('ingli', 2),
 ('emor', 2),
 ('multiverb', 2),
 ('awwwweee', 2),
 ('scuffletown', 2),
 ('widdup', 2),
 ('tuitalk', 2),
 ('foreheadwhen', 2),
 ('juraj', 2),
 ('bsdi', 2),
 ('luhhhhhhh', 2),
 ('darras', 2),
 ('ayyyyyyyyyyyyyyy', 2),
 ('owwwwwww', 2),
 ('parsheen', 2),
 ('pulihora', 2),
 ('planings', 2)]

# 6. data storage

In [90]:
# finally we have 2 versions of preprocessing of data so far
# first one is preprocessed train data without spelling and shorthand words correction, 
# but after assigning space, correction of common shorthand words, adding missing charactors, removal of stopwords, emoji, emoticon and punctuations
# the second one is preprocessed train data with spelling and shorthand words correction.
# and I also have extracted all the emoji and emoticons of the data, which can be later processed for further analysis

In [91]:
def save_data(name,data):
    pickle_file = open('DataSet\\'+name,mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [92]:

save_path6 = "pre_{a}_all_no_correct_{b}.pkl".format(a=path_change,b=version_change)
save_path3_9 = "pre_{a}_3turn_no_emo_{b}.pkl".format(a=path_change,b=version_change)
save_patha_9 = "pre_{a}_all_no_emo_{b}.pkl".format(a=path_change,b=version_change)
save_data(save_path6 ,text_all_6)
save_data(save_path3_9,text_3_9)
save_data(save_patha_9,text_all_9)